In [52]:
import geopandas as gpd
from src.data import gedi_pipeline
from src.data.fire_perimeters import FirePerimeters, FirePerimetersDB

import importlib
importlib.reload(gedi_pipeline)


2023-06-15 21:58:13,693 DEBUG: Logger /home/jk871/fire-regen/src/data/gedi_pipeline.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]


<module 'src.data.gedi_pipeline' from '/home/jk871/fire-regen/src/data/gedi_pipeline.py'>

## Running all stages of the GEDI pipeline

### Stage 2 - basic l4a processing

In [3]:
gedi = gedi_pipeline.stage_2_basic_processing_of_l4a_sierras_data(save=True)
gedi

2023-06-15 20:13:58,198 DEBUG: Saving stage 2 processed shots as pickle. [in stage_2_basic_processing_of_l4a_sierras_data at /home/jk871/fire-regen/src/data/gedi_pipeline.py:458]


,shot_number,longitude,latitude,agbd,agbd_pi_lower,agbd_pi_upper,agbd_se,beam_type,sensitivity,pft_class,gedi_year,gedi_month,absolute_time
0,184890800200086632,-117.983310,37.034945,19.594500,2.419877,53.254089,2.981980,full,0.959151,6,2022,3,2022-03-19 02:41:53.252278+00:00
1,184891100200081451,-117.985725,37.040705,17.788033,1.812951,50.249866,2.982302,full,0.966117,6,2022,3,2022-03-19 02:41:53.339050+00:00
2,184891100200081452,-117.985239,37.041043,16.219337,1.336309,47.591499,2.982735,full,0.966954,6,2022,3,2022-03-19 02:41:53.347314+00:00
3,184891100200081453,-117.984755,37.041380,6.750648,-9999.000000,29.955231,2.990220,full,0.970945,6,2022,3,2022-03-19 02:41:53.355578+00:00
4,184891100200081454,-117.984267,37.041720,4.370386,-9999.000000,24.677294,2.994677,full,0.961078,6,2022,3,2022-03-19 02:41:53.363842+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27629560,145740600300499241,-120.454065,40.766560,4.699281,-9999.000000,25.447033,2.993938,full,0.957520,6,2021,7,2021-07-09 13:28:27.680490+00:00
27629561,145740600300499242,-120.453532,40.766257,2.924632,-9999.000000,21.060156,2.998635,full,0.957790,6,2021,7,2021-07-09 13:28:27.688756+00:00
27629562,145740600300499244,-120.452463,40.765649,1.500722,-9999.000000,16.865877,3.004451,full,0.954659,6,2021,7,2021-07-09 13:28:27.705288+00:00
27629564,145740800300301239,-120.463172,40.762280,3.823108,-9999.000000,23.357908,2.996024,full,0.955201,6,2021,7,2021-07-09 13:28:27.978045+00:00


### Stage 3 - Match with MTBS burn raster

This can be done with a 3x3 or 2x2 kernel. 3x3 gives us better precision and guarantees data matching, while 2x2 has lower accuracy but preserves more data points.

In [5]:
# 3x3 kernel
gedi_3x3 = gedi_pipeline.stage_3_match_burn_raster_l4a_sierras(3)

2023-06-15 20:27:59,044 DEBUG: Saving stage 3 processed shots as pickle, for kernel 3. [in stage_3_match_burn_raster_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:440]


In [6]:
# 2x2 kernel
gedi_2x2 = gedi_pipeline.stage_3_match_burn_raster_l4a_sierras(2)

2023-06-15 20:38:19,290 DEBUG: Saving stage 3 processed shots as pickle, for kernel 2. [in stage_3_match_burn_raster_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:440]


### Stage 3B - Match with fire perimeters from CalFire

These include fires before 1984, as well as fires that are smaller than 1000 acres - which is around 4km^2. This may take a while because we're doing spatial joins between fire perimeters and 9 million GEDI shots.

In [18]:
# Get Fire perimeter data
sierras = gpd.read_file("../../data/shapefiles/sierras_convex_hull.shp")
firep_db = FirePerimetersDB('../../data/fire_perimeters.gdb/')
sierra_firep = FirePerimeters(firep_db).filter_within_geometry(sierras)

In [ ]:
#TODO: Consider doing this. For now, we won't because the processing time of geo 
# spatial matching between all of these fires will be time consuming. Instead, let's
# focus on fires > 1000 acres that are part of MTBS DB.

### Stage 4 - Filter based on matching with burn raster

For 3x3 shots, after filtering, we end up with 7,829,875 unburned shots, and 516,474 burned shots. Burned shots in this case include also the ones
that burned more than once.

In [31]:
# 3x3 kernel
gedi_3x3 = gedi_pipeline.stage_4_filter_burns_l4a_sierras(3)

2023-06-15 21:14:00,748 DEBUG: Read in intermediate data from stage 3. [in stage_4_filter_burns_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:358]
2023-06-15 21:14:35,947 DEBUG: Filter burn areas. [in stage_4_filter_burns_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:361]
2023-06-15 21:14:37,652 DEBUG: Excluded shots on the burn boundaries, shots remaining:         9178162 [in divide_shots_into_burned_and_unburned at /home/jk871/fire-regen/src/data/gedi_pipeline.py:413]
2023-06-15 21:14:37,947 DEBUG: Number of GEDI shots that burned at least once:                  1262136 [in divide_shots_into_burned_and_unburned at /home/jk871/fire-regen/src/data/gedi_pipeline.py:418]
2023-06-15 21:14:39,144 DEBUG: Number of GEDI shots that never burned since 1984:         7829875 [in divide_shots_into_burned_and_unburned at /home/jk871/fire-regen/src/data/gedi_pipeline.py:425]
2023-06-15 21:14:39,548 DEBUG: Number of GEDI shots that have a perfect match with burn  

For 2x2 shots, after filtering, we end up with 7,852,535 unburned shots, and 895,053 burned shots. Burned shots in this case include also the ones
that burned more than once.

In [32]:
# 2x2 kernel
gedi_2x2 = gedi_pipeline.stage_4_filter_burns_l4a_sierras(2)

2023-06-15 21:15:02,444 DEBUG: Read in intermediate data from stage 3. [in stage_4_filter_burns_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:358]
2023-06-15 21:15:36,383 DEBUG: Filter burn areas. [in stage_4_filter_burns_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:361]
2023-06-15 21:15:38,059 DEBUG: Excluded shots on the burn boundaries, shots remaining:         9446235 [in divide_shots_into_burned_and_unburned at /home/jk871/fire-regen/src/data/gedi_pipeline.py:413]
2023-06-15 21:15:38,397 DEBUG: Number of GEDI shots that burned at least once:                  1440937 [in divide_shots_into_burned_and_unburned at /home/jk871/fire-regen/src/data/gedi_pipeline.py:418]
2023-06-15 21:15:39,644 DEBUG: Number of GEDI shots that never burned since 1984:         7852535 [in divide_shots_into_burned_and_unburned at /home/jk871/fire-regen/src/data/gedi_pipeline.py:425]
2023-06-15 21:15:40,146 DEBUG: Number of GEDI shots that have a perfect match with burn  

### Stage 5 - filter for regrowth

Burned shots left - 436,817.

In [34]:
gedi_burned_3x3, gedi_unburned_3x3 = gedi_pipeline.stage_5_filter_for_regrowth_l4a_sierras(3)

2023-06-15 21:23:39,298 DEBUG: Number of shots that happened after fires:                    446058 [in filter_shots_for_regrowth_analysis at /home/jk871/fire-regen/src/data/gedi_pipeline.py:331]
2023-06-15 21:23:39,369 DEBUG: Number of shots that burned in 2-4 categories:                    436817 [in filter_shots_for_regrowth_analysis at /home/jk871/fire-regen/src/data/gedi_pipeline.py:336]
2023-06-15 21:23:39,378 DEBUG: Saving stage 5 processed burned shots as pkl,             for kernel 3. [in stage_5_filter_for_regrowth_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:310]
2023-06-15 21:23:40,000 DEBUG: Saving stage 5 processed unburned shots as pkl,             for kernel 3. [in stage_5_filter_for_regrowth_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:317]


Burned shots left - 755,887.

In [35]:
gedi_burned_2x2, gedi_unburned_2x2 = gedi_pipeline.stage_5_filter_for_regrowth_l4a_sierras(2)

2023-06-15 21:24:00,342 DEBUG: Number of shots that happened after fires:                    774368 [in filter_shots_for_regrowth_analysis at /home/jk871/fire-regen/src/data/gedi_pipeline.py:331]
2023-06-15 21:24:00,456 DEBUG: Number of shots that burned in 2-4 categories:                    755887 [in filter_shots_for_regrowth_analysis at /home/jk871/fire-regen/src/data/gedi_pipeline.py:336]
2023-06-15 21:24:00,471 DEBUG: Saving stage 5 processed burned shots as pkl,             for kernel 2. [in stage_5_filter_for_regrowth_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:310]
2023-06-15 21:24:01,445 DEBUG: Saving stage 5 processed unburned shots as pkl,             for kernel 2. [in stage_5_filter_for_regrowth_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:317]


### Stage 6 - Match with Land Cover

In [40]:
gedi_burned_3x3, gedi_unburned_3x3 = gedi_pipeline.stage_6_match_land_cover_l4a_sierras(3)

2023-06-15 21:33:49,375 DEBUG: Matching land cover for year 2017. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:33:53,573 DEBUG: Matching land cover for year 1996. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:33:55,485 DEBUG: Matching land cover for year 2004. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:33:57,418 DEBUG: Matching land cover for year 1995. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:33:59,024 DEBUG: Matching land cover for year 2015. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:34:01,550 DEBUG: Matching land cover for year 1991. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:34:03,495 DEBUG: Matching land cover for year 1985. [in match_

In [41]:
gedi_burned_2x2, gedi_unburned_2x2 = gedi_pipeline.stage_6_match_land_cover_l4a_sierras(2)

2023-06-15 21:41:33,877 DEBUG: Matching land cover for year 2017. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:41:37,398 DEBUG: Matching land cover for year 1996. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:41:38,190 DEBUG: Matching land cover for year 2004. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:41:38,820 DEBUG: Matching land cover for year 1995. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:41:39,243 DEBUG: Matching land cover for year 2015. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:41:41,038 DEBUG: Matching land cover for year 1991. [in match_burn_landcover at /home/jk871/fire-regen/src/data/gedi_raster_matching.py:88]
2023-06-15 21:41:41,367 DEBUG: Matching land cover for year 1985. [in match_

### Stage 7 - Filter based on Land Cover

For burned areas, we match each shot with the land cover as it was a year before the fire.

For unburned areas, we match each shot with the land cover from 2021.

Burned shots left - 225814, unburned left - 2707122.

In [47]:
gedi_burned_3x3, gedi_unburned_3x3 = gedi_pipeline.stage_7_filter_land_cover_l4a_sierras(3)

2023-06-15 21:51:31,791 DEBUG: Number of shots that were trees before they burned:                 225814 [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:240]
2023-06-15 21:51:33,124 DEBUG: Number of unburned shots that were trees in 2021:             2707122 [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:247]
2023-06-15 21:51:33,748 DEBUG: Saving stage 7 processed burned shots as pkl,             for kernel 3. [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:260]
2023-06-15 21:51:34,086 DEBUG: Saving stage 7 processed unburned shots as pkl,             for kernel 3. [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:267]


Interestingly, for filtering land cover we still use 3x3 kernel, even on 2x2 burn raster matches. This is due to cali forests being rather fragmented, so it's good to be extra certain about the forest cover.

Burned shots left -417824, unburned left - 2715511.

In [50]:
gedi_burned_2x2, gedi_unburned_2x2 = gedi_pipeline.stage_7_filter_land_cover_l4a_sierras(2)

2023-06-15 21:54:18,372 DEBUG: Number of shots that were trees before they burned:                 417824 [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:240]
2023-06-15 21:54:19,961 DEBUG: Number of unburned shots that were trees in 2021:             2715511 [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:247]
2023-06-15 21:54:20,824 DEBUG: Saving stage 7 processed burned shots as pkl,             for kernel 2. [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:260]
2023-06-15 21:54:21,660 DEBUG: Saving stage 7 processed unburned shots as pkl,             for kernel 2. [in stage_7_filter_land_cover_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:267]


### Stage 8 - Match Terrain

In [51]:
gedi_burned_3x3, gedi_unburned_3x3 = gedi_pipeline.stage_8_match_terrain_l4a_sierras(3)

2023-06-15 21:57:11,647 DEBUG: Saving stage 8 processed burned shots as pkl,             for kernel 3. [in stage_8_match_terrain_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:199]
2023-06-15 21:57:16,016 DEBUG: Saving stage 8 processed unburned shots as pkl,             for kernel 3. [in stage_8_match_terrain_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:206]


In [53]:
gedi_burned_2x2, gedi_unburned_2x2 = gedi_pipeline.stage_8_match_terrain_l4a_sierras(2)

2023-06-15 22:00:08,142 DEBUG: Saving stage 8 processed burned shots as pkl,             for kernel 2. [in stage_8_match_terrain_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:199]
2023-06-15 22:00:15,279 DEBUG: Saving stage 8 processed unburned shots as pkl,             for kernel 2. [in stage_8_match_terrain_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:206]


### Stage 9 - Match Terrain

In [54]:
gedi_burned_3x3, gedi_unburned_3x3 = gedi_pipeline.stage_9_filter_terrain_l4a_sierras(3)

2023-06-15 22:01:32,666 DEBUG: Saving stage 9 processed burned shots as pkl,             for kernel 3. [in stage_9_filter_terrain_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:159]
2023-06-15 22:01:33,110 DEBUG: Saving stage 9 processed unburned shots as pkl,             for kernel 3. [in stage_9_filter_terrain_l4a_sierras at /home/jk871/fire-regen/src/data/gedi_pipeline.py:166]


In [ ]:
gedi_burned_2x2, gedi_unburned_2x2 = gedi_pipeline.stage_9_filter_terrain_l4a_sierras(2)